# Building a ReAct Agent for Computational Chemistry

This notebook demonstrates how to build a **ReAct-style agent** using:

- **LangChain**
- **ALCF Inference Endpoint** (via `ChatOpenAI`)
- Three domain-specific tools:
  - `molecule_name_to_smiles`
  - `smiles_to_coordinate_file`
  - `run_mace_calculation`

The agent can:
1. Take a molecule name.
2. Convert it to a SMILES string.
3. Generate a coordinate file from the SMILES.
4. Run a MACE-based calculation on the structure.

> ⚠️ **Note:** Sometimes the agent may skip tool calls and answer from its internal knowledge. This is expected behavior in ReAct-style agents.


In [3]:
# Imports
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

from tools import (
    molecule_name_to_smiles,
    smiles_to_coordinate_file,
    run_mace_calculation,
)
from inference_auth_token import get_access_token


## Authenticate with the ALCF Inference Endpoint

We use a helper function `get_access_token()` (from `inference_auth_token.py`)  
to obtain an access token for the ALCF Inference Endpoint.


In [4]:
# Get access token

access_token = get_access_token()
print("Access token acquired.")

Access token acquired.


## Initialize the LLM via ALCF Inference Endpoint

We wrap the ALCF Inference Endpoint as a LangChain-compatible `ChatOpenAI` model.

- `model_name` is set to `openai/gpt-oss-120b`. Other available models can be found here: https://docs.alcf.anl.gov/services/inference-endpoints/#web-ui
- `api_key` is the access token you just retrieved.
- `base_url` points to the ALCF vLLM deployment.
- `temperature=0` for deterministic behavior during the tutorial.


In [5]:
# Initialize the ALCF Inference Endpoint model

llm = ChatOpenAI(
    model_name="openai/gpt-oss-120b",
    api_key=access_token,
    base_url="https://data-portal-dev.cels.anl.gov/resource_server/sophia/vllm/v1",
    temperature=0,
)

llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7fd2b4dbffd0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7fd2b4dbf880>, root_client=<openai.OpenAI object at 0x7fd2b4dbf340>, root_async_client=<openai.AsyncOpenAI object at 0x7fd2b4dbf100>, model_name='openai/gpt-oss-120b', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://data-portal-dev.cels.anl.gov/resource_server/sophia/vllm/v1')

## Register and Inspect Tools

We register the following tools with the agent:

1. `molecule_name_to_smiles(name: str) -> str`  
   Convert a molecule name (e.g., *benzene*) to a SMILES string.

2. `smiles_to_coordinate_file(smiles: str, output_file: str) -> str`  
   Convert a SMILES string to a coordinate file (e.g., `.xyz` or `.mol`).

3. `run_mace_calculation(input_file: str, mace_model_name: str, float, device: str, fmax: float, max_steps: int) -> Dict[str, Any]`  
   Run a MACE-based calculation using the provided structure file.

In this cell, we’ll just register the tools and print basic information about them.


In [6]:
# Register tools and inspect them

tools = [molecule_name_to_smiles, smiles_to_coordinate_file, run_mace_calculation]

for idx, tool in enumerate(tools):
    print(f"TOOL {idx}:")
    print(tool)
    print("-" * 80)


TOOL 0:
name='molecule_name_to_smiles' description='Convert a molecule name to SMILES format.\n\n    Parameters\n    ----------\n    name : str\n        The name of the molecule to convert.\n\n    Returns\n    -------\n    str\n        The SMILES string representation of the molecule.\n\n    Raises\n    ------\n    IndexError\n        If the molecule name is not found in PubChem.' args_schema=<class 'langchain_core.utils.pydantic.molecule_name_to_smiles'> func=<function molecule_name_to_smiles at 0x7fd3402ee440>
--------------------------------------------------------------------------------
TOOL 1:
name='smiles_to_coordinate_file' description='Convert a SMILES string to a coordinate file.\n\n    Parameters\n    ----------\n    smiles : str\n        SMILES string representation of the molecule.\n    output_file : str, optional\n        Path to save the output coordinate file (currently XYZ only).\n    randomSeed : int, optional\n        Random seed for RDKit 3D structure generation, by

## Build a ReAct-style Agent

We now create a ReAct-style agent using `create_agent` from `langchain.agents`.

The agent will:
- Decide when to call each tool,
- Observe the tool outputs,
- And synthesize a final answer for the user.


In [7]:
# Build the ReAct agent with the specified tools

agent = create_agent(llm, tools=tools)

In [8]:
prompt = "What are the SMILES strings of methanol and ethanol using your tools?"
for chunk in agent.stream(
    {"messages": prompt},
    stream_mode="values",
):
    new_message = chunk["messages"][-1]
    new_message.pretty_print()


================================ Human Message =================================

What are the SMILES strings of methanol and ethanol using your tools?
================================== Ai Message ==================================
Tool Calls:
  molecule_name_to_smiles (chatcmpl-tool-d91b7c311dfb4e76adad252630682b5e)
 Call ID: chatcmpl-tool-d91b7c311dfb4e76adad252630682b5e
  Args:
    name: methanol
================================= Tool Message =================================
Name: molecule_name_to_smiles

CO
================================== Ai Message ==================================
Tool Calls:
  molecule_name_to_smiles (chatcmpl-tool-7914b28fa9024795b3feaa3b1a48c52c)
 Call ID: chatcmpl-tool-7914b28fa9024795b3feaa3b1a48c52c
  Args:
    name: ethanol
================================= Tool Message =================================
Name: molecule_name_to_smiles

CCO
================================== Ai Message ==================================

The SMILES strings are:

- **Met

In [9]:
prompt = "Optimize the structure of a water molecule using MACE"
for chunk in agent.stream(
    {"messages": prompt},
    stream_mode="values",
):
    new_message = chunk["messages"][-1]
    new_message.pretty_print()


================================ Human Message =================================

Optimize the structure of a water molecule using MACE
================================== Ai Message ==================================
Tool Calls:
  molecule_name_to_smiles (chatcmpl-tool-5e1631a9616341a0848ae3ebf152b592)
 Call ID: chatcmpl-tool-5e1631a9616341a0848ae3ebf152b592
  Args:
    name: water
================================= Tool Message =================================
Name: molecule_name_to_smiles

O
================================== Ai Message ==================================
Tool Calls:
  molecule_name_to_smiles (chatcmpl-tool-cbb33c5503364e2098af011343ce0a0b)
 Call ID: chatcmpl-tool-cbb33c5503364e2098af011343ce0a0b
  Args:
    name: water
================================= Tool Message =================================
Name: molecule_name_to_smiles

O
================================== Ai Message ==================================
Tool Calls:
  smiles_to_coordinate_file (chatcmpl-tool-c7

/lus/eagle/projects/ALCFAITP/04-Inference-Workflows/env/_ai4s_agentic_conda/lib/python3.10/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.
Using Materials Project MACE for MACECalculator with /home/tdpham2/.cache/mace/20231210mace128L0_energy_epoch249model
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.


/lus/eagle/projects/ALCFAITP/04-Inference-Workflows/env/_ai4s_agentic_conda/lib/python3.10/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
Default dtype float32 does not match model dtype float64, converting models to float32.
      Step     Time          Energy          fmax
BFGS:    0 17:21:59      -14.039596        0.923767
BFGS:    1 17:22:02      -14.051286        0.131385
BFGS:    2 17:22:04      -14.051504        0.009730
================================= Tool Message =================================
Name: run_mace_calculation

{"status": "success", "message": "MACE geometry optimization completed.", "mode": "geometry_optimization", "converged": true, "input_file": "/lus/grand/projects/IQC/thang/ALCF_contributions/ai-science-training-series/04-Inference-Workflows/Agentic-workflows/water.xyz", "mace_model_name": "small", "device": "cpu", "final_energy_eV": -14.051504135131836, "final_positions": [[0.00609904506945858, 0.3925188947248634, 2.405221200838474e-23], [-0.7793843708624317, -0.18419662539154424, -7.072800693133495e-24], [0.7732853257826247, -0.20832227074423412, 0.0]],